In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
import collections
import codecs
import cerberus
import schema
import csv

In [2]:
OSM_FILE = "Seattlemap"  


In [3]:
import xml.etree.ElementTree as ET

SAMPLE_FILE = "sample.osm"

k = 10

def get_element(osm_file, tags=('node', 'way', 'relation')):
    
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

with open(SAMPLE_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')
    
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

# Tag Types

In [6]:
#Patters to search for tag attribute 'K' with lower alphabets, colons and problem characters
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
    

In [7]:
#Counting the lower, lower_colon and problem chars
def key_type(element, keys):
    if element.tag == "tag":
        
        l = lower.search(element.attrib['k'])
        lc = lower_colon.search(element.attrib['k'])
        pc = problemchars.search(element.attrib['k'])
        if l:
            keys["lower"] += 1
        elif lc:
            keys["lower_colon"] += 1
        elif pc:
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
        
        
    return keys





In [6]:
#defining keys dictionary
def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


In [7]:
#testing the count of lower , lower_colon and problem chars
def test():
    
    keys = process_map(SAMPLE_FILE)
    pprint.pprint(keys)
    
if __name__ == "__main__":
    test()

{'lower': 23465, 'lower_colon': 11199, 'other': 788, 'problemchars': 0}


## Typecasting

In [8]:
from audit import *

In [9]:
def test():

    users = process_map(SAMPLE_FILE)
    pprint.pprint(len(users))
   

In [10]:
if __name__ == "__main__":
    test()

<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
(

('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type '

<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
(

('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type '

<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
(

...

<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
(

('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type '

('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type '

<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
(

<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
('=>', <type 'str'>)
<type 'str'>
(

# Problems Encountered in the Map

In [56]:
# Improving Street Names and Postal Codes
from auditStreetNames import *

In [57]:
def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

In [58]:
st_types = audit(SAMPLE_FILE)
OUTPUT_FILE = "output.osm"
with open(OUTPUT_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')
    
    for i, elem in enumerate(get_element(SAMPLE_FILE)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    for st_type, st_names in st_types.iteritems():
                        for name in st_names:
                            better_name = update_name(name, mapping)
                            if tag.attrib['v'] == name:
                                tag.attrib['v'] = better_name
                if is_postal_code(tag) :
                    if len(tag.attrib['v']) > 5:
                        tag.attrib['v'] =  tag.attrib['v'][ 0:5]
        output.write(ET.tostring(elem, encoding='utf-8'))
    output.write('</osm>')
    

## Preparing for Database

In [59]:
from data import *


In [60]:
import codecs
import data
 
if __name__ == '__main__':    
    process_map(OUTPUT_FILE)
    

In [61]:
import sqlite3

In [62]:
db = sqlite3.connect("C:\sqlite-windows\sqlite_windows\seattle.db")
c = db.cursor()

In [63]:
query = "select * from nodes where user = 'Skybunny'"
c.execute(query)
rows = c.fetchall()
print(rows)

[(29459816, 47.5167515, -122.2912635, u'Skybunny', 1751737, 7, 22706626, u'2014-06-03T04:10:00Z\r'), (29461047, 47.5268948, -122.2998268, u'Skybunny', 1751737, 9, 22706626, u'2014-06-03T04:10:01Z\r'), (29978553, 47.3528573, -122.2957619, u'Skybunny', 1751737, 10, 44638507, u'2016-12-24T04:47:32Z\r'), (34279389, 47.3157794, -122.2453642, u'Skybunny', 1751737, 3, 25824316, u'2014-10-03T05:48:08Z\r'), (34707500, 47.2947875, -122.2904293, u'Skybunny', 1751737, 3, 54917373, u'2017-12-25T22:48:43Z\r'), (34711137, 47.3703223, -122.244256, u'Skybunny', 1751737, 5, 25824178, u'2014-10-03T05:34:11Z\r'), (34711160, 47.3730876, -122.244318, u'Skybunny', 1751737, 3, 25925261, u'2014-10-07T19:36:46Z\r'), (35436537, 47.3876688, -122.2432013, u'Skybunny', 1751737, 3, 25824212, u'2014-10-03T05:38:28Z\r'), (35436549, 47.3937477, -122.2349799, u'Skybunny', 1751737, 2, 25822717, u'2014-10-03T01:29:30Z\r'), (35442190, 47.4010555, -122.2241384, u'Skybunny', 1751737, 3, 25822717, u'2014-10-03T01:29:30Z\r'), 

# Problems in the map data

## Postal codes

In [65]:
query = "SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL \
         SELECT * FROM ways_tags) tags \
         WHERE tags.key='postcode'\
         GROUP BY tags.value\
         ORDER BY count DESC;"
c.execute(query)
rows = c.fetchall()

In [66]:
print(rows)

[(u'98178', 249), (u'98118', 114), (u'98057', 23), (u'98371', 19), (u'98092', 13), (u'98042', 11), (u'98032', 10), (u'98001', 4), (u'98030', 4), (u'98354', 4), (u'98010', 2), (u'98027', 2), (u'98038', 2), (u'98055', 2), (u'98058', 2), (u'98059', 2), (u'98372', 2), (u'98391', 2), (u'90092', 1), (u'98003', 1), (u'98032-1762', 1), (u'98057-4040', 1), (u'98104', 1), (u'98108', 1), (u'98168', 1), (u'98188', 1), (u'98198', 1), (u'98321', 1), (u'98373', 1), (u'98374', 1)]


## Sort Cities by count

In [67]:
query = "SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION  \
         SELECT * FROM ways_tags) tags\
         WHERE tags.key = 'city'\
         GROUP BY tags.value\
         ORDER BY count DESC;"
c.execute(query)
rows = c.fetchall()

In [68]:
print(rows)

[(u'Seattle', 365), (u'Renton', 29), (u'Kent', 25), (u'Puyallup', 24), (u'Auburn', 17), (u'Milton', 4), (u'Bonney Lake', 3), (u'Maple Valley', 3), (u'Black Diamond', 2), (u'Covington', 2), (u'Edgewood', 2), (u'Sumner', 2), (u'Algona', 1), (u'Buckley', 1), (u'Des Moines', 1), (u'Pacific', 1), (u'SeaTac', 1)]


# Overview of Data

File Sizes

Seattlemap.osm ......... 129 MB
sample.osm ............. 13 MB
output.osm ............. 13 MB
seattle.db ............. 7 MB
nodes.csv .............. 4 MB
nodes_tags.csv ......... 267 KB
ways.csv ............... 437 KB
ways_tags.csv .......... 974 KB
ways_nodes.cv .......... 1.7 MB  

### Number of Nodes

In [70]:
query = "select count(*) from nodes"
c.execute(query)
rows = c.fetchall()
print(rows)

[(56861,)]


### Number of Ways

In [71]:
query = "select count(*) from ways"
c.execute(query)
rows = c.fetchall()
print(rows)

[(7266,)]


### Number of Unique Users

In [72]:
query = "select count(distinct(e.uid)) from(select uid from nodes UNION ALL select uid from ways) e"
c.execute(query)
rows = c.fetchall()
print(rows)

[(436,)]


### Top 10 Contributing Users

In [73]:
query = "select e.user,count(*) as num from(select user from nodes UNION ALL select user from ways) as e GROUP BY e.user\
        ORDER BY num DESC\
        LIMIT 10"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'Omnific', 19989), (u'Grauer Elefant', 4687), (u'Amoebabadass', 4468), (u'STBrenden', 3977), (u'Glassman', 3680), (u'csytsma', 2836), (u'woodpeck_fixbot', 2403), (u'Geodesy99', 2245), (u'Skybunny', 1766), (u'zephyr', 1414)]


### Number of Users Appearing only once(having 1 post)

In [74]:
query = "SELECT COUNT(*) FROM (SELECT e.user, COUNT(*) as num\
        FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e\
        GROUP BY e.user HAVING num=1)  u;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(107,)]


# Additional Ideas

### Top 10 Appearing Amenities

In [75]:
query = "SELECT value, COUNT(*) as num FROM nodes_tags\
        WHERE key='amenity'\
        GROUP BY value\
        ORDER BY num DESC\
        LIMIT 10;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'restaurant', 42), (u'bench', 38), (u'fast_food', 34), (u'waste_basket', 20), (u'cafe', 19), (u'bank', 15), (u'doctors', 15), (u'fuel', 13), (u'dentist', 12), (u'school', 12)]


### Biggest Religion

In [76]:
query = "SELECT nodes_tags.value, COUNT(*) as num\
         FROM nodes_tags \
         JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i\
         ON nodes_tags.id=i.id \
         WHERE nodes_tags.key='religion' \
         GROUP BY nodes_tags.value \
         ORDER BY num DESC \
         LIMIT 1;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'christian', 6)]


### Most Popular Cuisines

In [77]:
query = "SELECT nodes_tags.value, COUNT(*) as num \
        FROM nodes_tags \
        JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i \
        ON nodes_tags.id=i.id \
        WHERE nodes_tags.key='cuisine' \
        GROUP BY nodes_tags.value \
        ORDER BY num DESC;"
c.execute(query)
rows = c.fetchall()
print(rows)

[(u'pizza', 7), (u'japanese', 4), (u'chinese', 3), (u'asian', 2), (u'burger', 2), (u'mexican', 2), (u'thai', 2), (u'american;savory_pancakes;pancake;breakfast', 1), (u'barbecue', 1), (u'burger;asian', 1), (u'indian', 1), (u'italian', 1), (u'pancake;breakfast', 1), (u'vietnamese', 1)]


##  ideas about the dataset

### Solution 
 if the dataset is an API , we can get the live data everytime we call the API and can do the transformations such as improving street names, updating postal codes much easier
### Benefits:
1. Ease of code transformation such as using inbuilt libraries Json 
2. less complexity of code
3. by using API, the JSON data can be easily converted into Python Dictionaries 
### Anticipated Issues:
1. Runtime might be slow because of continous calls to API  while developing the code
2. API Key generation related issues
